In [43]:
import requests
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
# Just query with london.
url = 'https://www.numbeo.com/api/city_prices?api_key=b6vniprdzkppy7&query=London'
resp = requests.get(url)
print(resp.status_code)

In [3]:
json = resp.json()
json

{'name': 'London, United Kingdom',
 'currency': 'GBP',
 'monthLastUpdate': 3,
 'contributors': 728,
 'yearLastUpdate': 2019,
 'prices': [{'data_points': 98,
   'item_id': 1,
   'lowest_price': 10,
   'average_price': 15,
   'highest_price': 20,
   'item_name': 'Meal, Inexpensive Restaurant, Restaurants'},
  {'data_points': 90,
   'item_id': 2,
   'lowest_price': 40,
   'average_price': 50,
   'highest_price': 70,
   'item_name': 'Meal for 2 People, Mid-range Restaurant, Three-course, Restaurants'},
  {'data_points': 79,
   'item_id': 3,
   'lowest_price': 5,
   'average_price': 6,
   'highest_price': 7,
   'item_name': 'McMeal at McDonalds (or Equivalent Combo Meal), Restaurants'},
  {'data_points': 70,
   'item_id': 4,
   'lowest_price': 4,
   'average_price': 5,
   'highest_price': 5,
   'item_name': 'Domestic Beer (0.5 liter draught), Restaurants'},
  {'data_points': 65,
   'item_id': 5,
   'lowest_price': 4,
   'average_price': 4.5,
   'highest_price': 5,
   'item_name': 'Imported 

In [42]:
df = json_normalize(json)
df.head()

,city_id,contributors,currency,monthLastUpdate,name,prices,yearLastUpdate
0,6512,728,GBP,3,"London, United Kingdom","[{'data_points': 98, 'item_id': 1, 'lowest_pri...",2019


In [5]:
prices_df = json_normalize(json['prices'])
prices_df.head()

,average_price,data_points,highest_price,item_id,item_name,lowest_price
0,15.0,98,20.0,1,"Meal, Inexpensive Restaurant, Restaurants",10.0
1,50.0,90,70.0,2,"Meal for 2 People, Mid-range Restaurant, Three...",40.0
2,6.0,79,7.0,3,McMeal at McDonalds (or Equivalent Combo Meal)...,5.0
3,5.0,70,5.0,4,"Domestic Beer (0.5 liter draught), Restaurants",4.0
4,4.5,65,5.0,5,"Imported Beer (0.33 liter bottle), Restaurants",4.0


In [34]:
new_df = prices_df
new_df['city_name'] = df['name'].values[0]
new_df['currency'] = df['currency'].values[0]
new_df['contributors'] = df['contributors'].values[0]
new_df['city_id'] = df['city_id'].values[0]
new_df['lastUpdated'] = str(df['monthLastUpdate'].values[0]) + '/' + str(df['yearLastUpdate'].values[0])

In [41]:
reorder_attrs = ['city_id','city_name', 'item_id', 'item_name', 
 'lowest_price', 'highest_price', 'average_price', 
 'currency', 'contributors','data_points']
new_df = new_df[reorder_attrs]

In [63]:
# Make a function out of this cleaning to apply per each city.
def jsonCityPricesToDF(json):
    df = json_normalize(json)
    prices_df = json_normalize(json['prices'])
    new_df = prices_df
    new_df['city_name'] = df['name'].values[0]
    new_df['currency'] = df['currency'].values[0]
    new_df['contributors'] = df['contributors'].values[0]
    new_df['city_id'] = df['city_id'].values[0]
    new_df['lastUpdated'] = str(df['monthLastUpdate'].values[0]) + '/' + str(df['yearLastUpdate'].values[0])
    reorder_attrs = ['city_id','city_name', 'item_id', 'item_name', 'lowest_price', 'highest_price', 'average_price', 'currency', 'contributors','data_points']
    return new_df[reorder_attrs]

def getPrices(city):
    url = 'https://www.numbeo.com/api/city_prices?api_key=b6vniprdzkppy7&query={}'.format(city)
    resp = requests.get(url)
    if resp.status_code != 200:
        raise Exception(resp.status_code)
    else:
        return resp.json()

In [154]:
# Load cities of interest
citiesOfInterest = pd.read_csv('DataVisual_Cities.csv', sep=';')
citiesOfInterest['to EUR'] = citiesOfInterest['to EUR'].str.replace(',','.')
citiesOfInterest.head()

,Cantidad,CITY_NAME,Unnamed: 2,GLOBAL,EUROPE,BUSINESS INSIDER,Currency,to EUR,Population
0,1,Zurich,Switzerland,1,1,2.0,CHF,0.88,402.762
1,2,Geneva,Switzerland,1,1,6.0,CHF,0.88,198.979
2,3,Basel,Switzerland,1,1,8.0,CHF,0.88,171.017
3,4,Bern,Switzerland,1,1,11.0,CHF,0.88,133.115
4,5,Amsterdam,Netherland,2,2,9.0,EUR,1,821.752


In [157]:
# Query price items for all cities of interest.
i = 0
df
for city in citiesOfInterest['CITY_NAME']:
    json = getPrices(city)
    cityPricesDF = jsonCityPricesToDF(json)
    cityPricesDF['toEur'] = citiesOfInterest['to EUR'].values[i]
    cityPricesDF.toEur.apply(lambda x: float(x))
    if i == 0:
        df = cityPricesDF
    else:
        df = pd.concat([df, cityPricesDF], sort=False)
    i+=1

In [185]:
df.city_name.value_counts()

Reykjavik, Iceland        55
Oslo, Norway              55
Copenhagen, Denmark       55
Zurich, Switzerland       55
Dublin, Ireland           55
Lisbon, Portugal          55
Prague, Czech Republic    55
Nuremberg, Germany        55
Brussels, Belgium         55
Helsinki, Finland         55
Ljubljana, Slovenia       55
Munich, Germany           55
Berlin, Germany           55
Bern, Switzerland         55
Basel, Switzerland        55
Luxembourg, Luxembourg    55
Stockholm, Sweden         55
Dusseldorf, Germany       55
Hamburg, Germany          55
London, United Kingdom    55
Brno, Czech Republic      55
Barcelona, Spain          55
Stuttgart, Germany        55
Rome, Italy               55
Milan, Italy              55
Paris, France             55
Frankfurt, Germany        55
Vienna, Austria           55
Lyon, France              55
Geneva, Switzerland       55
Amsterdam, Netherlands    55
Madrid, Spain             55
Tallinn, Estonia          55
Name: city_name, dtype: int64

In [186]:
df

,city_id,city_name,item_id,item_name,lowest_price,highest_price,average_price,currency,contributors,data_points,toEur
0,6379,"Zurich, Switzerland",1,"Meal, Inexpensive Restaurant, Restaurants",20.000000,30.000000,25.000000,CHF,321,37,0.88
1,6379,"Zurich, Switzerland",2,"Meal for 2 People, Mid-range Restaurant, Three...",70.000000,120.000000,100.000000,CHF,321,37,0.88
2,6379,"Zurich, Switzerland",3,McMeal at McDonalds (or Equivalent Combo Meal)...,12.000000,17.000000,14.000000,CHF,321,28,0.88
3,6379,"Zurich, Switzerland",4,"Domestic Beer (0.5 liter draught), Restaurants",5.000000,8.000000,7.000000,CHF,321,31,0.88
4,6379,"Zurich, Switzerland",5,"Imported Beer (0.33 liter bottle), Restaurants",5.000000,8.000000,6.000000,CHF,321,29,0.88
5,6379,"Zurich, Switzerland",6,"Coke/Pepsi (0.33 liter bottle), Restaurants",3.000000,5.000000,4.053571,CHF,321,28,0.88
6,6379,"Zurich, Switzerland",7,"Water (0.33 liter bottle) , Restaurants",2.500000,5.000000,3.634615,CHF,321,26,0.88
7,6379,"Zurich, Switzerland",8,"Milk (regular), (1 liter), Markets",1.500000,2.000000,1.742500,CHF,321,20,0.88
8,6379,"Zurich, Switzerland",9,"Loaf of Fresh White Bread (500g), Markets",2.000000,4.000000,2.704545,CHF,321,22,0.88
9,6379,"Zurich, Switzerland",11,"Eggs (regular) (12), Markets",4.000000,8.000000,5.336522,CHF,321,23,0.88
